This file is to run manually a head to head evaluation

In [ ]:
import copy
from agent_configs.cfr_config import CFRConfig
from active_player import ActivePlayer
from cfr_agent import CFRAgent
import torch
from cfr_network import CFRNetwork
from cfr_utils import evaluatebots, WrapperEnv, load_agents


fhp = pyspiel.load_game("universal_poker", {"numPlayers":2, "numSuits": 4, "numRanks":13, "numHoleCards": 2, "numBoardCards": "0 3", "bettingAbstraction": "fcpa", "numRounds":2, "blind": "50 100"})
leduc = pyspiel.load_game("universal_poker", {"numPlayers":2, "numSuits": 2, "numRanks":3, "numHoleCards": 1, "numBoardCards": "0 1", "bettingAbstraction": "fcpa", "numRounds":2, "blind": "50 100"})
leducconfig = {"state_representation_size": 16}
fhpconfig = {"state_representation_size": 108}
leducgame = WrapperEnv(leduc)
fhpgame = WrapperEnv(fhp)

chosen_game = 'leduc'

hidden_dim = 128
input_dim = 16 if chosen_game == 'leduc' else 108
output_dim = 4
num_players = 2
replay_buffer_size = 4000000
minibatch_size = 10000
steps_per_epoch = 3000
traversals = 3000
training_steps = 20000
lr = 0.0001
optimizer = None

p_v_networks = {'input_shape':input_dim, 'output_shape':output_dim, 'hidden_size':hidden_dim, 'learning_rate':lr, 'optimizer':optimizer}
active_player_obj = ActivePlayer(num_players)
path1 = 'checkpoints/CFR_LEDUC_Full/policy/notlinear/2027909/CFR_LEDUC_Full_68.pt'
path2 = 'checkpoints/CFR_LEDUC_Full/policy/notlinear/4021243/CFR_LEDUC_Full_134.pt'
config = CFRConfig(
    config_dict={'network': {'policy': p_v_networks, 'value': p_v_networks, 'num_players':num_players},
                 'replay_buffer_size':replay_buffer_size,
                 'minibatch_size':minibatch_size,
                 'steps_per_epoch':steps_per_epoch,
                 'traversals': traversals,
                 'training_steps': training_steps,
                 'active_player_obj': active_player_obj,
                 },
    game_config={'num_players':num_players,
                 'observation_space':input_dim,
                 'action_space':4,},
)

agent1, agent2 = load_agents(path1, path2, p_v_networks, num_players)

In [ ]:

p1, p2 = evaluatebots(agent1, agent2, 100000, leduc_env, config, input_dim)
print("Player 1: " + str(np.mean(p1)/100) + "mbb/game")
print("Player 2: " + str(np.mean(p2)/100) + "mbb/game")